# PiLiDAR

## 2D Visualization

In [3]:
import cv2
import numpy as np
import threading
import time
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, Image


class Plot2D:
    def __init__(self, imgsize=(500, 500), xlim=(-2500, 2500), backend="opencv", pixel=True):
        self.backend = backend.lower()
        self.pixel = pixel
        self.imgsize = imgsize
        self.xlim = xlim
        self.points = []
        self.points_lock = threading.Lock()
        self.enable = [True]

        if self.backend == "matplotlib":
            self._initialize_matplotlib()

        self.visualization_thread = threading.Thread(target=self.update_plot)
        self.visualization_thread.start()

    def _initialize_matplotlib(self):
        self.figsize = (self.imgsize[0] // 72, self.imgsize[1] // 72)
        self.fig, self.ax = plt.subplots(figsize=self.figsize)

    def update(self, new_points):
        with self.points_lock:
            self.points.clear()
            self.points.extend(new_points)

    def update_plot(self):
        while self.enable[0]:
            with self.points_lock:
                if self.points:
                    current_points = np.array(self.points)
                else:
                    continue

            if self.backend == "opencv":
                self._update_opencv(current_points)
            elif self.backend == "matplotlib":
                self._update_matplotlib(current_points)

    def _update_opencv(self, points):
        image = np.zeros((self.imgsize[1], self.imgsize[0], 1), dtype=np.uint8)
        
        # Filter out invalid points
        valid_mask = np.isfinite(points[:, 0]) & np.isfinite(points[:, 1]) & np.isfinite(points[:, 2])
        points = points[valid_mask]
        
        # # Calculate aspect ratio
        aspect_ratio = self.imgsize[0] / self.imgsize[1]
        
        # Normalize points based on aspect ratio
        x = ((points[:, 0] - self.xlim[0]) / (self.xlim[1] - self.xlim[0]) * self.imgsize[0]).astype(int)
        y = ((points[:, 1] - self.xlim[0]) / (self.xlim[1] - self.xlim[0]) * self.imgsize[0]).astype(int)
        
        # TODO: Adjust y to center around the middle of the image height
        y = - int(self.imgsize[1]/2) + y  # aspect_ratio
        
        # Create a mask to filter points within the valid range
        mask = (x >= 0) & (x < self.imgsize[0]) & (y >= 0) & (y < self.imgsize[1])
        x = x[mask]
        y = y[mask]
        luminance = points[:, 2][mask]
        
        # Ensure luminance values are within the valid range for uint8
        luminance = np.clip(luminance, 0, 255).astype(np.uint8)
        
        for i in range(len(x)):
            if self.pixel:
                image[y[i], x[i]] = luminance[i]
            else:
                cv2.circle(image, (x[i], y[i]), 1, int(luminance[i]), -1)
        
        image = cv2.rotate(image, cv2.ROTATE_180)

        _, buffer = cv2.imencode('.jpg', image)
        img_str = buffer.tobytes()
        clear_output(wait=True)
        display(Image(data=img_str))
        time.sleep(0.02)

    def _update_matplotlib(self, points):
        clear_output(wait=True)
        plt.close('all')
        self.fig, self.ax = plt.subplots(figsize=self.figsize)
        self.ax.clear()
        x = points[:, 0]
        y = points[:, 1]
        luminance = points[:, 2] / 255
        self.ax.scatter(x, y, c=luminance, cmap='gray', s=0.5)
        self.ax.set_xlim(self.xlim[0], self.xlim[1])
        self.ax.set_ylim(self.xlim[0], self.xlim[1])
        self.ax.set_facecolor('black')
        self.fig.patch.set_facecolor('black')
        plt.axis('off')
        plt.show()

    def close(self):
        self.enable[0] = False
        self.visualization_thread.join()
        print("Visualization stopped.")



In [2]:
### Visualization example

# # PROXY DATA
# def read_lidar(max):
#     pointcount = 2000
#     points = np.random.rand(pointcount, 2) * max - max/2 
    
#     # print("X", points[:, 0].min(), points[:, 0].max())
#     # print("Y", points[:, 1].min(), points[:, 1].max())

#     luminance = np.random.randint(0, 256, size=(pointcount, 1), dtype=np.uint8)
#     points = np.hstack((points, luminance))
#     return points


# # Start the visualization
# plot_2d = Plot2D(imgsize=(600,400), xlim=(-3500,3500), backend="opencv", pixel=True)

# # Main loop
# cycles = 100
# try:
#     for i in range(cycles):
#         points = read_lidar(4500)
#         plot_2d.update(points)
#         time.sleep(0.02)
# except KeyboardInterrupt:
#     plot_2d.close()

# plot_2d.close()

## Lidar

In [ ]:
from lib.config import Config
from lib.a4988_driver import A4988
from lib.lidar_driver import Lidar


def print_inline(text):
    print(f"\r{text}", end='')

target_resolution = 0.5

config = Config()
config.init()
config.update_target_res(target_resolution)


plot_2d = Plot2D(imgsize=(800,400), xlim=(-6000,6000), backend="opencv", pixel=True)


# initialize stepper
stepper = A4988(config.get("STEPPER", "pins", "DIR_PIN"), 
                config.get("STEPPER", "pins", "STEP_PIN"), 
                config.get("STEPPER", "pins", "MS_PINS"), 
                delay=config.get("STEPPER", "STEP_DELAY"),
                step_angle=config.get("STEPPER", "STEP_ANGLE"),
                microsteps=config.get("STEPPER", "MICROSTEPS"),
                gear_ratio=config.get("STEPPER", "GEAR_RATIO"))

# initialize lidar
lidar = Lidar(config, visualization=None)

# callback function for lidar.read_loop()
def move_steps_callback():
    stepper.move_steps(config.steps if config.SCAN_ANGLE > 0 else -config.steps)
    lidar.z_angle = stepper.get_current_angle()

    # print_inline(f"z-angle: {round(lidar.z_angle, 2)}")
    plot_2d.update(lidar.points_2d)

try:
    lidar.read_loop(callback=move_steps_callback, max_packages=config.max_packages)

except KeyboardInterrupt:
    plot_2d.close()

plot_2d.close()


In [4]:
from lib.pointcloud import save_raw_scan, get_scan_dict

# Save raw_scan to pickle file
raw_scan = get_scan_dict(lidar.z_angles, cartesian_list=lidar.cartesian_list)
save_raw_scan(lidar.raw_path, raw_scan)

# 3D

In [ ]:
import os

# from lib.config import Config
from lib.pointcloud import process_raw, load_pointcloud, downsample

# scan_id = "240824-1230"
# config = Config()
# config.init(scan_id=scan_id)

if os.path.exists(config.pcd_path):
    # load PLY file
    pcd = load_pointcloud(config.pcd_path)

else:
    # DISABLE TEXTURING AND FILTERING
    config.set(True, "ENABLE_3D")
    config.set(False, "ENABLE_PANO")
    config.set(False, "ENABLE_VERTEXCOLOUR")
    config.set(False, "ENABLE_FILTERING")

    # load raw scan from config.raw_path
    pcd = process_raw(config, save=True)

low_pcd = downsample(pcd, voxel_size=1)

## VISUALIZATION
https://github.com/isl-org/Open3D/tree/main/docs/tutorial/visualization  
https://github.com/isl-org/Open3D/tree/main/docs/jupyter

### Plotly:

In [ ]:
import plotly.graph_objects as go
import numpy as np

# replaces open3d.visualization.draw_plotly 
def draw_plotly(pcd, width=800, height=600, point_size=1, background_color='white'):
    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors)

    fig = go.Figure(data=[go.Scatter3d(
        x=points[:, 0], y=points[:, 1], z=points[:, 2],
        mode='markers', marker=dict(size=point_size, color=colors))])

    ax = dict(showgrid=False, backgroundcolor=background_color, showticklabels=False)
    scene = dict(aspectmode='data', xaxis=ax, yaxis=ax, zaxis=ax)

    fig.update_layout(scene=scene, width=width, height=height)
    fig.show()

# Example usage with custom window size and background color
draw_plotly(low_pcd, width=1000, height=800, background_color='white')

In [ ]:
### regular Open3D Visualizer (requires a Window Manager):

# from lib.visualization import visualize
# visualize(low_pcd, view="front", unlit=True, point_size=1, fullscreen=True)


### Open3D Web-Visualizer:

# from open3d.web_visualizer import draw
# draw([low_pcd], title="PiLiDAR", width=800, height=600, bg_color=(0.2,0.2,0.2, 1), point_size=2)  # lookat, eye, up